In [1]:
'''
!pip install datasets

!pip install transformers 
!pip install sentencepiece

!pip install sacrebleu
'''

'\n!pip install datasets\n\n!pip install transformers \n!pip install sentencepiece\n\n!pip install sacrebleu\n'

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from torch.nn.parallel import DataParallel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
model = DataParallel(model)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
new_tokens = ['<H>', '<R>', '<T>']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)

tokenizer.add_tokens("[MASK]")
tokenizer.mask_token = "[MASK]"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

In [78]:
class WebNLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            triples = item['original_triple_sets']['otriple_set']
            input_text = self.prefix
            for outer_list in triples: 
                for triple in outer_list:
                    triple_txt = triple.split("|")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except (KeyError, IndexError):
            print("1")
            print(item['original_triple_sets']['otriple_set'])
            print(item['original_triple_sets']['otriple_set'][0])
            print(triples)
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['lex']['text'][0]
        except (KeyError, IndexError):
            print("2")
            print(item)
            #print(item['original_triple_sets']['otriple_set'])
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [5]:
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.module.config.max_length = MAX_TARGET_LENGTH

# set up the device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

DataParallel(
  (module): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): Linear(

In [7]:
# load the WebNLG dataset
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['train']

Generating train split:   0%|          | 0/6940 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4615 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/webnlg_challenge_2017/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Adaptive pretraining

For STA, we fine-tuned the PLMs on a small amount of labeled data from the target task using a maximum likelihood estimation (MLE) objective. This involves training the model to maximize the likelihood of generating the correct output given the input graph and labeled data. This process helps to further adapt the PLM to the specific requirements of the target task and improve its performance on that task.

In [7]:
import random

pretrain_texts = []
for sample in dataset:
    try:
        text = sample['lex']['text'][0]
        pretrain_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs = tokenizer(pretrain_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

pretrain_data = torch.utils.data.TensorDataset(input_ids, attention_mask)

pretrain_loader = torch.utils.data.DataLoader(pretrain_data, batch_size=int(60), shuffle=True)

pretrain_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
pretrain_criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

pretrain_epochs = 2  # Set the number of pre-training epochs
masking_prob = 0.15  # Probability of masking a token

if tokenizer.mask_token is None:
    # Manually set a mask token if not already defined
    tokenizer.add_tokens("[MASK]")
    tokenizer.mask_token = "[MASK]"
    tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

for epoch in range(pretrain_epochs):
    running_loss = 0.0
    for inputs, attention_mask in pretrain_loader:
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        batch_size, seq_length = inputs.shape
        
        # Create a mask for randomly selected tokens
        mask = torch.rand(inputs.shape) < masking_prob
        
        # Randomly replace selected tokens with [MASK] token
        masked_inputs = inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id
        
        pretrain_optimizer.zero_grad()
        outputs = model(input_ids=masked_inputs, attention_mask=attention_mask, decoder_input_ids=inputs)
        
        # Compute the loss only for the masked tokens
        masked_logits = outputs.logits[mask]
        masked_labels = inputs[mask]
        loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))
        
        loss.backward()
        pretrain_optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(pretrain_data)
    print(f"Pretrain Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_loss:.4f}")


Pretrain Epoch 1/2 - loss: 3.2586
Pretrain Epoch 2/2 - loss: 0.6002


For LMA, we first fine-tuned the PLMs on a small amount of task-specific data using a masked language modeling objective. This involves randomly masking some tokens in the input sequence and training the model to predict the masked tokens based on the context provided by the unmasked tokens. This process helps to adapt the PLM to the specific characteristics of the target task and improve its performance on that task.

# Finetuning

In [9]:
# set up the data loader
train_data = WebNLGDataset(dataset)
batch_size = 32 #16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [74]:
# set up the optimizer and the loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) #3e-5
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)


In [11]:
from tqdm import tqdm

# fine-tune the model
num_epochs = 2
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")


  0%|          | 0/217 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 217/217 [01:47<00:00,  2.02it/s]


Epoch 1/2 - loss: 0.8518


100%|██████████| 217/217 [01:44<00:00,  2.09it/s]

Epoch 2/2 - loss: 0.6766


In [7]:
class DartDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            tripleset = item['tripleset']
            input_text = self.prefix
            for triple in tripleset: 
                triple_txt = triple
                input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except (KeyError, IndexError):
            print("1")
            print(item['tripleset'])
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['annotations']['text'][0]
        except (KeyError, IndexError):
            print("2")
            print(item)
            target_text = ""
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

        #print decoding of input and target
        #print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
        #print(tokenizer.decode(target_ids[0], skip_special_tokens=True))
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [72]:
# set up the data loader
dataset_dart = load_dataset('dart', split='train', ignore_verifications=True)
train_data = DartDataset(dataset_dart)
batch_size = 32 #16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [75]:
from tqdm import tqdm

# fine-tune the model
num_epochs = 2
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")

  0%|          | 0/954 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 32%|███▏      | 309/954 [02:33<05:21,  2.01it/s]


KeyboardInterrupt: 

In [8]:
class E2ENLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "Translate from Meaning Representation to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input meaning representation
        try:
            input_text = self.prefix + item['meaning_representation']
        except KeyError:
            print("Failed to access 'meaning_representation' for item at index: ", idx)
            input_text = self.prefix

        # preprocess the target text
        try:
            target_text = item['human_reference']
        except KeyError:
            print("Failed to access 'human_reference' for item at index: ", idx)
            target_text = ""

        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

        # print decoded input and target texts
        #print("Input Text: ", tokenizer.decode(input_ids[0], skip_special_tokens=True))
        #print("Target Text: ", tokenizer.decode(target_ids[0], skip_special_tokens=True))

        return input_ids.squeeze(0), target_ids.squeeze(0)


In [16]:
dataset_e2e = load_dataset('e2e_nlg', split='train')
train_data = E2ENLGDataset(dataset_e2e)
batch_size = 32 #16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/42061 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4672 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4693 [00:00<?, ? examples/s]

Dataset e2e_nlg downloaded and prepared to /root/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430. Subsequent calls will reuse this data.


In [17]:
from tqdm import tqdm

# fine-tune the model
num_epochs = 2
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")

100%|██████████| 1315/1315 [10:22<00:00,  2.11it/s]


Epoch 1/2 - loss: 1.0168


100%|██████████| 1315/1315 [10:21<00:00,  2.12it/s]

Epoch 2/2 - loss: 0.9699


In [25]:
# Save the entire model
torch.save(model, 'model_T5_flan_small_88')
print("Model saved successfully.")

Model saved successfully.


In [105]:
# Load the model
model = torch.load('/kaggle/input/models/model_T5_flan_small_88')

# Print a confirmation message
print("Model loaded successfully.")

Model loaded successfully.


In [97]:
# Load the model
model = torch.load('/kaggle/input/models/model_T5_flan_small_epoch3')

# Print a confirmation message
print("Model loaded successfully.")

Model loaded successfully.


## are we accounting for the multiple texts targets in the bleu? it doesn't look like it

In [5]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.8 MB/s eta 0:00:00


In [10]:
batch_size=32

In [106]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm


# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']] # filter out samples with empty targets 
# Select a subset of the validation dataset
#subset_size = 10  # Choose the desired subset size
#validation_subset = sample(list(validation_dataset), subset_size)
validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)


  0%|          | 0/3 [00:00<?, ?it/s]

In [107]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])
    
bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, multiple_references)

print(f"BLEU score: {bleu.score}")
print(f"BLEU score with multiple references: {bleu_multiple.score}")

BLEU score: 0.41761210976679014
BLEU score with multiple references: 88.9139705019462


In [108]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])
    
# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references)

# Then pad all sublists to that length
padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references]
    
bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, padded_references)

print(f"BLEU score: {bleu.score}")
print(f"BLEU score with padded references: {bleu_multiple.score}")

BLEU score: 0.41761210976679014
BLEU score with padded references: 45.68530592006045


In [109]:
from datasets import load_metric

metric = load_metric('sacrebleu')

# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references)

# Then pad all sublists to that length
padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references]

# Now 'padded_references' is a list of lists, where each sublist has the same length.
# We can now compute the SacreBLEU score.

# Note the change in the compute line
score = metric.compute(predictions=predictions, references = padded_references)

print(f"SacreBLEU score: {score['score']}")


SacreBLEU score: 27.663688366094107


In [110]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
chrf_multiple = corpus_chrf(predictions, multiple_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 43.707786114425126
CHR F++ score with multiple references: 92.18986336629665


In [111]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
chrf_multiple = corpus_chrf(predictions, padded_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 43.707786114425126
CHR F++ score with multiple references: 63.2961977222028


In [120]:
from datasets import load_metric
import numpy as np


metric = load_metric('bertscore')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

# Compute the score
score = metric.compute(predictions=predictions, references=references, lang='en')

print(f"BERTScore: {np.mean(score['precision'])}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.9076188037579942


In [112]:
i=5
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Politician', 'size': 1, 'eid': 'Id6', 'original_triple_sets': {'otriple_set': [['Abdul_Taib_Mahmud | successor | Sulaiman_Abdul_Rahman_Taib']]}, 'modified_triple_sets': {'mtriple_set': [['Abdul_Taib_Mahmud | successor | Sulaiman_Abdul_Rahman_Taib']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good', 'good'], 'lid': ['Id1', 'Id2', 'Id3'], 'text': ["Abdul Taib Mahmud's successor was Sulaiman Abdul Rahman Taib.", 'Abdul Taib Mahmud was succeded by Sulaiman Abdul Rahman Taib.', 'The sucessor to Abdul Taib Mahmud was Sulaiman Abdul Rahman Taib.'], 'lang': ['', '', '']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
[['Abdul_Taib_Mahmud | successor | Sulaiman_Abdul_Rahman_Taib']]
Abdul-Taib-Mahmud's successor was Abdul-Taib-Mahmud who was succeeded by Sulaiman Abdul-Rahman Taib.
["Abdul Taib Mahmud's successor was Sulaiman Abdul Rahman Taib.", 'Abdul Taib Mahmud was succeded by Sulaiman Abdul Rahman Taib.', 'The sucessor to Abd

In [113]:
i=10
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Politician', 'size': 1, 'eid': 'Id11', 'original_triple_sets': {'otriple_set': [['Abner_W._Sibal | deathPlace | Alexandria,_Virginia']]}, 'modified_triple_sets': {'mtriple_set': [['Abner_W._Sibal | deathPlace | Alexandria,_Virginia']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good', 'good'], 'lid': ['Id1', 'Id2', 'Id3'], 'text': ['Abner W Sibal died in Alexandria, Virginia.', 'Abner W. Sibal died in Alexandria, Virginia.', 'Abner W Sibal died in Alexandria, Virginia.'], 'lang': ['', '', '']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
[['Abner_W._Sibal | deathPlace | Alexandria,_Virginia']]
Abner W. Sibal died in Alexandria, Virginia.
['Abner W Sibal died in Alexandria, Virginia.', 'Abner W. Sibal died in Alexandria, Virginia.', 'Abner W Sibal died in Alexandria, Virginia.']


In [114]:
i=50
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Politician', 'size': 1, 'eid': 'Id51', 'original_triple_sets': {'otriple_set': [['United_States_Army | battles | Spanish–American_War'], ['United_States_Army | battle | Spanish–American_War']]}, 'modified_triple_sets': {'mtriple_set': [['United_States_Army | battles | Spanish–American_War']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good'], 'lid': ['Id1'], 'text': ['The United States Army was involved in battles in the Spanish-American War.'], 'lang': ['']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
[['United_States_Army | battles | Spanish–American_War'], ['United_States_Army | battle | Spanish–American_War']]
The United States Army fought in the Spanish-American War.
['The United States Army was involved in battles in the Spanish-American War.']


In [115]:
i=0
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Politician', 'size': 1, 'eid': 'Id1', 'original_triple_sets': {'otriple_set': [['Aaron_S._Daggett | award | Purple_Heart']]}, 'modified_triple_sets': {'mtriple_set': [['Aaron_S._Daggett | award | Purple_Heart']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good'], 'lid': ['Id1', 'Id2'], 'text': ['Aaron S Daggett was awarded the Purple Heart.', 'Aaron S. Daggett was awarded the Purple Heart.'], 'lang': ['', '']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
[['Aaron_S._Daggett | award | Purple_Heart']]
Aaron S. Daggett was awarded the Purple Heart.
['Aaron S Daggett was awarded the Purple Heart.', 'Aaron S. Daggett was awarded the Purple Heart.']


### known error, sometimes the prompt leaks into the output 

In [116]:
i=70
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'MeanOfTransportation', 'size': 1, 'eid': 'Id71', 'original_triple_sets': {'otriple_set': [['Alfa_Romeo_164 | relatedMeanOfTransportation | Lancia_Thema'], ['Alfa_Romeo_164 | related | Lancia_Thema']]}, 'modified_triple_sets': {'mtriple_set': [['Alfa_Romeo_164 | relatedMeanOfTransportation | Lancia_Thema']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good'], 'lid': ['Id1', 'Id2'], 'text': ['Alfa Romeo 164 and Lancia Thema are related types of transportation.', 'The related transport to the Alfa Romeo 164 is the Lancia Thema.'], 'lang': ['', '']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
[['Alfa_Romeo_164 | relatedMeanOfTransportation | Lancia_Thema'], ['Alfa_Romeo_164 | related | Lancia_Thema']]
The Alfa Romeo 164 and Lancia Thema are related means of transport.
['Alfa Romeo 164 and Lancia Thema are related types of transportation.', 'The related transport to the Alfa Romeo 164 is the Lancia Thema.']


In [117]:
i=130
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Athlete', 'size': 1, 'eid': 'Id131', 'original_triple_sets': {'otriple_set': [['Aleksandr_Prudnikov | team | FC_Amkar_Perm'], ['Aleksandr_Prudnikov | clubs | FC_Amkar_Perm']]}, 'modified_triple_sets': {'mtriple_set': [['Aleksandr_Prudnikov | club | FC_Amkar_Perm']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good'], 'lid': ['Id1', 'Id2'], 'text': ['Aleksandr Prudnikov plays for FC Amkar Perm.', 'Aleksandr Prudnikov plays for the FC Amkar Perm football club.'], 'lang': ['', '']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
[['Aleksandr_Prudnikov | team | FC_Amkar_Perm'], ['Aleksandr_Prudnikov | clubs | FC_Amkar_Perm']]
Aleksandr Prudnikov's club is FC Amkar Perm.
['Aleksandr Prudnikov plays for FC Amkar Perm.', 'Aleksandr Prudnikov plays for the FC Amkar Perm football club.']


In [118]:
i=1861
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Astronaut', 'size': 7, 'eid': 'Id971', 'original_triple_sets': {'otriple_set': [['William_Anders | dateOfRet | "1969-09-01"^^xsd:date', 'William_Anders | selection | 1963', 'William_Anders | timeInSpace | "8820.0"^^<http://dbpedia.org/datatype/minute>', 'William_Anders | birthDate | "1933-10-17"^^xsd:date', 'William_Anders | occupation | Fighter_pilot', 'William_Anders | birthPlace | British_Hong_Kong', 'William_Anders | mission | Apollo_8']]}, 'modified_triple_sets': {'mtriple_set': [['William_Anders | dateOfRetirement | "1969-09-01"', 'William_Anders | was selected by NASA | 1963', 'William_Anders | timeInSpace | "8820.0"(minutes)', 'William_Anders | birthDate | "1933-10-17"', 'William_Anders | occupation | Fighter_pilot', 'William_Anders | birthPlace | British_Hong_Kong', 'William_Anders | was a crew member of | Apollo_8']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good', 'good'], 'lid': ['Id1', 'Id2', 'Id3'], 'text': ['Test pilot William Anders was

In [119]:
i=1860
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(multiple_references[i])

{'category': 'Astronaut', 'size': 7, 'eid': 'Id970', 'original_triple_sets': {'otriple_set': [['William_Anders | dateOfRet | "1969-09-01"^^xsd:date', 'William_Anders | mission | Apollo_8', 'William_Anders | nationality | United_States', 'William_Anders | birthPlace | British_Hong_Kong', 'Apollo_8 | crew2Up | Buzz_Aldrin', 'Apollo_8 | crewMembers | Frank_Borman', 'Apollo_8 | operator | NASA']]}, 'modified_triple_sets': {'mtriple_set': [['William_Anders | dateOfRetirement | "1969-09-01"', 'William_Anders | was a crew member of | Apollo_8', 'William_Anders | nationality | United_States', 'William_Anders | birthPlace | British_Hong_Kong', 'Apollo_8 | backup pilot | Buzz_Aldrin', 'Apollo_8 | crewMembers | Frank_Borman', 'Apollo_8 | operator | NASA']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good', 'good'], 'lid': ['Id1', 'Id2', 'Id3'], 'text': ["William Anders was born in British Hong Kong and is a U.S Citizen. William was a member of the Apollo 8 crew (along with Frank

## there is a problem with empty target samples in the test set, we still need to check multiple triples!

In [15]:
from transformers import T5Tokenizer
from datasets import load_dataset

# Define the tokenizer
#tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the WebNLG dataset
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
dataset = [sample for sample in dataset if sample['lex']['text']]

# Create an instance of WebNLGDataset
webnlg_dataset = WebNLGDataset(dataset)

# Define the index of the example you want to test
example_index = 70

# Get the input and target texts for the example at the specified index
input_text, target_text = webnlg_dataset[example_index]

# Decode the input and target texts using the tokenizer
decoded_input_text = tokenizer.decode(input_text, skip_special_tokens=True)
decoded_target_text = tokenizer.decode(target_text, skip_special_tokens=True)

# Print the preprocessed input and target texts
print("Input Text:", decoded_input_text)
print("Target Text:", decoded_target_text)


  0%|          | 0/3 [00:00<?, ?it/s]

{'category': 'MeanOfTransportation', 'size': 1, 'eid': 'Id71', 'original_triple_sets': {'otriple_set': [['Alfa_Romeo_164 | relatedMeanOfTransportation | Lancia_Thema'], ['Alfa_Romeo_164 | related | Lancia_Thema']]}, 'modified_triple_sets': {'mtriple_set': [['Alfa_Romeo_164 | relatedMeanOfTransportation | Lancia_Thema']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good'], 'lid': ['Id1', 'Id2'], 'text': ['Alfa Romeo 164 and Lancia Thema are related types of transportation.', 'The related transport to the Alfa Romeo 164 is the Lancia Thema.'], 'lang': ['', '']}, 'test_category': 'testdata_unseen_with_lex', 'dbpedia_links': [], 'links': []}
translate from Graph to Text:  <H> Alfa_Romeo_164  <R>  relatedMeanOfTransportation  <T>  Lancia_Thema <H> Alfa_Romeo_164  <R>  related  <T>  Lancia_Thema
Input Text: translate from Graph to Text: Alfa_Romeo_164 relatedMeanOfTransportation Lancia_Thema Alfa_Romeo_164 related Lancia_Thema
Target Text: Alfa Romeo 164 and Lancia Thema are

## seeing how many empty targets there are in the testing set

In [18]:
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
count_empty_text = 0
for sample in dataset:
    if not sample['lex']['text']:
        count_empty_text += 1

print(f"Number of samples with empty 'lex' 'text' field: {count_empty_text}")


  0%|          | 0/3 [00:00<?, ?it/s]

Number of samples with empty 'lex' 'text' field: 1862


In [19]:
total_samples = len(dataset)
print(f"Total number of samples in the test dataset: {total_samples}")

Total number of samples in the test dataset: 4615


----

### testing the other datasets

In [13]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 32
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.module.config.max_length = MAX_TARGET_LENGTH


# load the WebNLG validation dataset
validation_dataset = load_dataset('dart', split='test', ignore_verifications=True)
# Select a subset of the validation dataset
validation_data = DartDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)

In [18]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])
 
bleu = corpus_bleu(predictions, references)

print(f"BLEU score: {bleu.score}")

BLEU score: 0.2051450723153009


In [68]:
from datasets import load_metric

metric = load_metric('sacrebleu')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

list_references = []
for reference in references:
    list_references.append([reference])

# Note the change in the compute line
score = metric.compute(predictions=predictions, references=list_references)

print(f"SacreBLEU score: {score['score']}")


SacreBLEU score: 24.493754509617915


In [19]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
print(f"CHR F++ score: {chrf.score}")

CHR F++ score: 48.48215141572602


In [27]:
!pip install -U nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [49]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')

# Assumptions:
# predictions is a list of predicted sentences
# references is a list of reference sentences

assert len(predictions) == len(references), "The number of predictions and references should be the same."

scores = []

for prediction, reference in zip(predictions, references):
    tokenized_prediction = word_tokenize(prediction)
    tokenized_reference = word_tokenize(reference)
    score = nltk.translate.meteor_score.meteor_score([tokenized_reference], tokenized_prediction)
    scores.append(score)

average_meteor_score = sum(scores) / len(scores)
print(f"Average METEOR score: {average_meteor_score}")


AttributeError: module 'sacrebleu' has no attribute 'sentence_meteor'

In [59]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [62]:
from datasets import load_metric
import numpy as np


metric = load_metric('bertscore')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

# Compute the score
score = metric.compute(predictions=predictions, references=references, lang='en')

print(f"BERTScore: {np.mean(score['precision'])}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.9684976935386658, 0.9517239928245544, 0.9555596113204956, 0.9669597744941711, 0.7615500092506409, 0.9741414785385132, 0.9090425968170166, 0.8719844818115234, 0.9122937321662903, 0.8878229856491089, 0.8318129777908325, 0.7686228156089783, 0.9176633954048157, 0.8045493960380554, 0.9807978868484497, 0.8816131353378296, 0.7751502394676208, 0.7214384078979492, 0.7524235844612122, 0.7952421307563782, 0.8292425274848938, 0.9255465269088745, 0.8995858430862427, 0.9131088256835938, 0.9289765357971191, 0.8034697771072388, 0.9093977212905884, 0.9233434796333313, 0.9341667890548706, 0.8790757656097412, 0.9166616201400757, 0.8572903871536255, 0.9376908540725708, 0.9291410446166992, 0.8428069949150085, 0.7945224046707153, 0.9067594408988953, 0.8201577067375183, 0.7635658383369446, 0.9075868725776672, 0.956684947013855, 0.9564284086227417, 0.9725723266601562, 0.9804766774177551, 0.8941009044647217, 0.8961852788925171, 0.9903538227081299, 0.9466882944107056, 0.9880983233451

In [67]:
print(f"BERTScore: {np.mean(score['precision'])}")

BERTScore: 0.924785583012708


In [57]:
import datasets
datasets.list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'character',
 'charcut_mt',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'mape',
 'mase',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'nist_mt',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'r_squared',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'smape',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics',
 'BucketHeadP65/confusion_matrix',
 'BucketHeadP65/roc_curve',
 'Drunper/metrica_tesi',
 'Felipehonorato/eer',
 'He-Xingwei/sari_metric',
 'JP-SystemsX/nDCG',
 'Josh98/nl2bash_m',
 'Kyle1668/squad',
 'Muennighoff/code_eval',
 'NCSOFT/harim_plus',
 'Natooz/ece',
 'NikitaMartynov/spell-check-met

In [46]:
pip install pyter3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [47]:
import pyter

# Assumptions:
# predictions is a list of predicted sentences
# references is a list of reference sentences

assert len(predictions) == len(references), "The number of predictions and references should be the same."

scores = []

for prediction, reference in zip(predictions, references):
    score = pyter.ter(reference.split(), prediction.split())
    scores.append(score)

average_ter_score = sum(scores) / len(scores)
print(f"Average TER score: {average_ter_score}")


Average TER score: 1.0390399552765
